In [68]:
# import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import glob
import csv
import os
import random

In [189]:
# Access data

def get_data(data_dir, labels_path):
    data_filenames = glob.glob(os.path.join(data_dir, '*npy'))
    random.shuffle(data_filenames)

    num_examples = len(data_filenames)

    max_length = 0
    for i, df in enumerate(data_filenames):
        max_length = max(max_length, np.load(df).shape[0])
    
    labels_dict = {}
    get_labels_dict(labels_path, labels_dict)
    
    X = np.zeros([num_examples, max_length])
    Y = np.zeros(num_examples)
    for i, df in enumerate(data_filenames):
        data = np.load(df)
        X[i, :data.shape[0]] = data
        
        label_key = df.split('/')[-1].split('.')[0].split('_')[0]
        Y[i] = labels_dict[label_key]

    # Convert -1 labels to 0
    Y[np.where(Y == -1)] = 0
        
    return X.astype(np.float32), Y.astype(np.float32)


def get_random_data(data_dir, labels_path, num_examples):
    data_filenames = glob.glob(os.path.join(data_dir, '*npy'))
    random.shuffle(data_filenames)

    max_length = 0
    for i in range(num_examples):
        df = data_filenames[i]
        max_length = max(max_length, np.load(df).shape[0])
    
    labels_dict = {}
    get_labels_dict(labels_path, labels_dict)
    
    X = np.zeros([num_examples, max_length])
    Y = np.zeros(num_examples)
    for i in range(num_examples):
        df = data_filenames[i]
        
        data = np.load(df)
        X[i, :data.shape[0]] = data
        
        label_key = df.split('/')[-1].split('.')[0].split('_')[0]
        Y[i] = labels_dict[label_key]

    # Convert -1 labels to 0
    Y[np.where(Y == -1)] = 0
        
    return X.astype(np.float32), Y.astype(np.float32)


def get_random_spect_data(data_dir, labels_path, num_examples=None):
    data_filenames = glob.glob(os.path.join(data_dir, '*npy'))
    random.shuffle(data_filenames)

    max_width = 0
    max_length = 0
    if num_examples is None:
        for i, df in enumerate(data_filenames):
            max_width = max(max_width, np.load(df).shape[0])
            max_length = max(max_length, np.load(df).shape[1])
        num_examples = i + 1
    else:
        for i in range(num_examples):
            df = data_filenames[i]
            max_width = max(max_width, np.load(df).shape[0])
            max_length = max(max_length, np.load(df).shape[1])
    
    labels_dict = {}
    get_labels_dict(labels_path, labels_dict)
    
    X = np.zeros([num_examples, max_length, max_width])
    Y = np.zeros(num_examples)
    for i in range(num_examples):
        df = data_filenames[i]
        
        data = np.load(df)
        X[i, :data.shape[1], :data.shape[0]] = data.T
        
        label_key = df.split('/')[-1].split('.')[0].split('_')[0]
        Y[i] = labels_dict[label_key]

    # Convert -1 labels to 0
    Y[np.where(Y == -1)] = 0
        
    return X.astype(np.float32), Y.astype(np.float32)


def get_labels_dict(reference_path, reference):
    with open(reference_path) as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in spamreader:
            reference[row[0]] = row[1]


In [70]:
# Methods to account for variable sequence lengths (call for each batch)
def length(sequence):
    used = tf.sign(tf.reduce_max(tf.abs(sequence), 2))
    length = tf.reduce_sum(used, 1)
    length = tf.cast(length, tf.int32)
    return length

def last_relevant(output, length):
    batch_size = tf.shape(output)[0]
    max_length = tf.shape(output)[1]
    out_size = int(output.get_shape()[2])
    index = tf.range(0, batch_size) * max_length + (length - 1)
    flat = tf.reshape(output, [-1, out_size])
    relevant = tf.gather(flat, index)
    
    return relevant

In [71]:

def _variable_on_cpu(name, shape, initializer):
    """
    Helper to create a Variable stored on CPU memory.
    """
    
    with tf.device('/cpu:0'):
        var = tf.get_variable(
            name=name, 
            shape=shape, 
            dtype=tf.float32, 
            initializer=initializer)

    return var

def _variable_with_weight_decay(name, shape, stddev, wd):
    """
    Helper to create an initialized Variable with weight decay.
    """

    var = _variable_on_cpu(
        name=name,
        shape=shape,
        initializer=tf.truncated_normal_initializer(
            stddev=stddev,
            dtype=tf.float32))

    if wd is not None:
        weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, 
                                   name='weight_loss')
        tf.add_to_collection('losses', weight_decay)

    return var


def loss(unscale_logits, labels):
    """
    Add L2Loss to all the trainable variables.
    """

    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=labels,
        logits=unscale_logits,
        name='cross_entropy')

    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')

    tf.add_to_collection('losses', cross_entropy_mean)

    total_loss = tf.add_n(tf.get_collection('losses'), name='total_loss')

    return total_loss

In [185]:
def rnn(features):
    
    features = features['x']
    
    # Input Layer
    input_layer = tf.reshape(features, [-1, 100, 151])

    # RNN layer #1
    with tf.variable_scope(name_or_scope='rnn1') as scope:
        n_units = 500
#         base_cell = tf.nn.rnn_cell.BasicRNNCell(n_units, activation=tf.nn.relu) tf.nn.rnn_cell.BasicLSTMCell
#         rnn_cell = tf.contrib.rnn.MultiRNNCell([tf.nn.rnn_cell.BasicRNNCell(n_units, activation=tf.nn.relu) for _ in range(2)])
        rnn_cell = tf.contrib.rnn.MultiRNNCell([tf.nn.rnn_cell.BasicLSTMCell(n_units) for _ in range(2)])

        outputs, rnn_state =  tf.nn.dynamic_rnn(rnn_cell, 
                                           inputs=input_layer,
                                           # initial_state=initial_state,
                                           dtype=tf.float32,
                                           # sequence_length=length(input_layer),
                                           time_major=False,
                                           scope=scope.name)
        
        print(outputs)
        
    # Fully connected layer #1
    with tf.variable_scope(name_or_scope='fc1') as scope:

        weights = _variable_with_weight_decay(
            name='weights',
            shape=[n_units, 250], # needs to match output dimension of RNN
            stddev=0.04,
            wd=0.004)

        biases = _variable_on_cpu(
            name='biases',
            shape=[250], # needs to match output dimension of RNN
            initializer=tf.constant_initializer(0.1))

        pre_activation = tf.add(tf.matmul(outputs[:, -1, :], weights), biases)

        fc1 = tf.nn.relu(
            features=pre_activation,
            name=scope.name)
        
    # Fully connected layer #2
    with tf.variable_scope(name_or_scope='fc2') as scope:

        weights = _variable_with_weight_decay(
            name='weights',
            shape=[250, 2], # needs to match output dimension of RNN
            stddev=0.04,
            wd=0.004)

        biases = _variable_on_cpu(
            name='biases',
            shape=[2], # needs to match output dimension of RNN
            initializer=tf.constant_initializer(0.1))

        pre_activation = tf.add(tf.matmul(fc1, weights), biases)

        fc2 = tf.nn.relu(
            features=pre_activation,
            name=scope.name)

    return fc2

# def train_neural_network(): 
#     prediction = recurrent_neural_network(x_placeholder)
#     cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=tf.reshape(y_placeholder, [batch_size, n_classes])))
#     optimizer = tf.train.AdamOptimizer().minimize(cost)
#     epoch_batch_itr = 10
    
#     correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y_placeholder, 1))
#     accuracy = tf.reduce_mean(tf.cast(correct, 'float32'))
        
#     with tf.Session() as sess:
#         tf.global_variables_initializer().run()

#         for epoch in range(hm_epochs):
#             epoch_loss = 0
#             for b in range(epoch_batch_itr):
#                 batchX, batchY = generate_batch(train_path)
#                 _, c = sess.run([optimizer, cost], feed_dict={x_placeholder: batchX, y_placeholder: batchY})
#                 epoch_loss += c
#             validation_batchX, validation_batchY = generate_batch(validation_path) 
#             print('Epoch', epoch, 'loss:', epoch_loss, 'Validation Accuracy:', accuracy.eval({x_placeholder: validation_batchX, y_placeholder: validation_batchY}))        

In [176]:

def rnn_model_fn(features, labels, mode):

    """
    Build model.
    """
    unscale_logits = rnn(features)
    
    # Generate predictions for PREDICT and EVAL modes.
    predictions = {
        'classes': tf.argmax(input=unscale_logits, axis=1),
        'probabilities': tf.nn.softmax(unscale_logits, name='softmax_tensor')
    }
    
    ####################
    # PREDICT
    ####################
    if mode == tf.estimator.ModeKeys.PREDICT: 
        return tf.estimator.EstimatorSpec(mode=mode, 
                                          predictions=predictions['probabilities'])
    
    else:
        # Calculate loss for both TRAIN and EVAL modes.
        labels = tf.cast(labels, tf.int64)
        total_loss = loss(unscale_logits, labels) #REFORM THIS LOSS FUNCTION

        # Add summary operation for total loss visualizaiton.
        tf.summary.scalar(
            name='total_loss',
            tensor=total_loss)
        
        
        ####################
        # TRAIN
        ####################
        if mode == tf.estimator.ModeKeys.TRAIN:
            
            # Compute gradients using Gradient Descent Optimizer.
            optimizer = tf.train.AdamOptimizer()

            grads_vars = optimizer.compute_gradients(loss=total_loss)

            # Add summary operations for gradient visualizations.
            for grad, var in grads_vars:
                if grad is not None:
                    tf.summary.histogram(
                        name=var.op.name + '/gradients', 
                        values=grad)

            train_op = optimizer.minimize(
                loss=total_loss,
                global_step=tf.train.get_global_step())

            # Add evaluation metrics for TRAIN mode.
            accuracy_train = tf.metrics.accuracy(
                labels=labels, 
                predictions=predictions["classes"])

            # Add summary operation for training accuracy visualizaiton.
            tf.summary.scalar(
                name='accuracy_train',
                tensor=accuracy_train[0])

            train_summary_hook = tf.train.SummarySaverHook(
                save_steps=10,
                output_dir='models/rnn',
                summary_op=tf.summary.merge_all())

            return tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss, 
                train_op=train_op,
                training_hooks=[train_summary_hook])
        
        
        ####################
        # EVALUATE
        ####################
        else:
            accuracy_valid = tf.metrics.accuracy(
                labels=labels, 
                predictions=predictions["classes"])

            # Add summary operation for validation accuracy visualizaiton.
            tf.summary.scalar(
                name='accuracy_validation',
                tensor=accuracy_valid[0])

            eval_metric_ops = {"accuracy": accuracy_valid}

            eval_summary_hook = tf.train.SummarySaverHook(
                save_steps=1,
                output_dir='models/rnn',
                summary_op=tf.summary.merge_all())

            return tf.estimator.EstimatorSpec(
                mode=mode, 
                loss=total_loss, 
                eval_metric_ops=eval_metric_ops,
                training_hooks=[eval_summary_hook])
        

In [197]:
# Main function for building, training and evaluating model.
def main(train_data, train_labels, eval_data, eval_labels, test_data):
    
    estimator_dir = 'models/rnn'
    
    # Delete directory containing events logs and checkpoints if it exists.
    if tf.gfile.Exists(estimator_dir):
        tf.gfile.DeleteRecursively(estimator_dir)

    # Create directory containing events logs and checkpoints.
    tf.gfile.MakeDirs(estimator_dir)
    
    # Create the Estimator.
    classifier = tf.estimator.Estimator(
        model_fn=rnn_model_fn, 
        model_dir=estimator_dir)

    for _ in range(100):
        
        # Train the model.
        train_input_fn = tf.estimator.inputs.numpy_input_fn(
            x={"x": train_data},
            y=train_labels,
            batch_size=128,
            num_epochs=None,
            shuffle=True)

        classifier.train(
            input_fn=train_input_fn,
            steps=50)

        # Evaluate the model and print results.
        eval_input_fn = tf.estimator.inputs.numpy_input_fn(
            x={"x": eval_data},
            y=eval_labels,
#             batch_size=,
            num_epochs=1,
            shuffle=False)
    
        eval_results = classifier.evaluate(input_fn=eval_input_fn)
        print(eval_results)
        
    # Generate predictions on test set.
    predict_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": test_data},
      num_epochs=1,
      shuffle=False)

    predictions = np.array(list(classifier.predict(input_fn=predict_input_fn))).T
    
    return predictions
        

In [178]:
# data_dir = 'data/sequence'

# X_train, Y_train = get_data(os.path.join(data_dir, 'training/sub'), 
#                             os.path.join(data_dir, 'REFERENCE.csv'))
                            
# X_valid, Y_valid = get_data(os.path.join(data_dir, 'validation/sub'), 
#                             os.path.join(data_dir, 'REFERENCE.csv'))

In [195]:
# Load training data

data_dir = '/Volumes/light/deeplearning_proj/data/spectrogram/training/sub_ws300'
labels_path = '/Volumes/light/deeplearning_proj/data/spectrogram/training/REFERENCE.csv'
num_examples = 1000

X_train, Y_train = get_random_spect_data(data_dir, labels_path, num_examples)

# Load validation data
data_dir = '/Volumes/light/deeplearning_proj/data/spectrogram/validation/sub_ws300'
labels_path = '/Volumes/light/deeplearning_proj/data/spectrogram/validation/REFERENCE.csv'

X_valid, Y_valid = get_random_spect_data(data_dir, labels_path)

In [198]:
pred = main(X_train, Y_train, X_valid, Y_valid, X_valid)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models/rnn', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
Tensor("rnn1/rnn1/transpose:0", shape=(128, 100, 500), dtype=float32)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into models/rnn/model.ckpt.
INFO:tensorflow:loss = 0.976333, step = 1
INFO:tensorflow:Saving checkpoints for 50 into models/rnn/model.ckpt.
INFO:tensorflow:Loss for final step: 0.317587.
Tensor("rnn1/rnn1/transpose:0", shape=(?, 100, 500), dtype=float32)
INFO:tensorflow:Starting evaluation at 2017-12-07-05:25:05
INFO:tensorflow:Restoring parameters from models/rnn/model.ckpt-50
INFO:tensorflow:Finished evaluation at 2017-12-07-05:25:15
INFO:tensorflow:Saving dict for global step 50: accuracy = 0.60424, glo

Tensor("rnn1/rnn1/transpose:0", shape=(?, 100, 500), dtype=float32)
INFO:tensorflow:Starting evaluation at 2017-12-07-06:00:50
INFO:tensorflow:Restoring parameters from models/rnn/model.ckpt-500
INFO:tensorflow:Finished evaluation at 2017-12-07-06:01:00
INFO:tensorflow:Saving dict for global step 500: accuracy = 0.635159, global_step = 500, loss = 1.32756
{'accuracy': 0.63515902, 'loss': 1.3275553, 'global_step': 500}
Tensor("rnn1/rnn1/transpose:0", shape=(128, 100, 500), dtype=float32)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from models/rnn/model.ckpt-500
INFO:tensorflow:Saving checkpoints for 501 into models/rnn/model.ckpt.
INFO:tensorflow:loss = 0.0367262, step = 501
INFO:tensorflow:Saving checkpoints for 550 into models/rnn/model.ckpt.
INFO:tensorflow:Loss for final step: 0.0446708.
Tensor("rnn1/rnn1/transpose:0", shape=(?, 100, 500), dtype=float32)
INFO:tensorflow:Starting evaluation at 2017-12-07-06:04:29
INFO:tensorflow:Restoring paramete

INFO:tensorflow:loss = 0.0287203, step = 951
INFO:tensorflow:Saving checkpoints for 1000 into models/rnn/model.ckpt.
INFO:tensorflow:Loss for final step: 0.0241995.
Tensor("rnn1/rnn1/transpose:0", shape=(?, 100, 500), dtype=float32)
INFO:tensorflow:Starting evaluation at 2017-12-07-06:37:15
INFO:tensorflow:Restoring parameters from models/rnn/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-12-07-06:37:26
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.614841, global_step = 1000, loss = 1.53076
{'accuracy': 0.61484098, 'loss': 1.53076, 'global_step': 1000}
Tensor("rnn1/rnn1/transpose:0", shape=(128, 100, 500), dtype=float32)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from models/rnn/model.ckpt-1000
INFO:tensorflow:Saving checkpoints for 1001 into models/rnn/model.ckpt.
INFO:tensorflow:loss = 0.0394776, step = 1001
INFO:tensorflow:Saving checkpoints for 1050 into models/rnn/model.ckpt.
INFO:tensorflow:Loss for final step: 0

Tensor("rnn1/rnn1/transpose:0", shape=(128, 100, 500), dtype=float32)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from models/rnn/model.ckpt-1450
INFO:tensorflow:Saving checkpoints for 1451 into models/rnn/model.ckpt.
INFO:tensorflow:loss = 0.0114804, step = 1451
INFO:tensorflow:Saving checkpoints for 1500 into models/rnn/model.ckpt.
INFO:tensorflow:Loss for final step: 0.00917552.
Tensor("rnn1/rnn1/transpose:0", shape=(?, 100, 500), dtype=float32)
INFO:tensorflow:Starting evaluation at 2017-12-07-07:13:23
INFO:tensorflow:Restoring parameters from models/rnn/model.ckpt-1500
INFO:tensorflow:Finished evaluation at 2017-12-07-07:13:34
INFO:tensorflow:Saving dict for global step 1500: accuracy = 0.636926, global_step = 1500, loss = 1.77249
{'accuracy': 0.63692582, 'loss': 1.7724924, 'global_step': 1500}
Tensor("rnn1/rnn1/transpose:0", shape=(128, 100, 500), dtype=float32)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters f

INFO:tensorflow:Finished evaluation at 2017-12-07-07:46:04
INFO:tensorflow:Saving dict for global step 1950: accuracy = 0.642226, global_step = 1950, loss = 1.92408
{'accuracy': 0.64222616, 'loss': 1.9240837, 'global_step': 1950}
Tensor("rnn1/rnn1/transpose:0", shape=(128, 100, 500), dtype=float32)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from models/rnn/model.ckpt-1950
INFO:tensorflow:Saving checkpoints for 1951 into models/rnn/model.ckpt.
INFO:tensorflow:loss = 0.00685397, step = 1951
INFO:tensorflow:Saving checkpoints for 2000 into models/rnn/model.ckpt.
INFO:tensorflow:Loss for final step: 0.0491715.
Tensor("rnn1/rnn1/transpose:0", shape=(?, 100, 500), dtype=float32)
INFO:tensorflow:Starting evaluation at 2017-12-07-07:49:31
INFO:tensorflow:Restoring parameters from models/rnn/model.ckpt-2000
INFO:tensorflow:Finished evaluation at 2017-12-07-07:49:42
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.613074, global_step = 2000, lo

...

INFO:tensorflow:Restoring parameters from models/rnn/model.ckpt-2900
INFO:tensorflow:Saving checkpoints for 2901 into models/rnn/model.ckpt.
INFO:tensorflow:loss = 0.00680903, step = 2901
INFO:tensorflow:Saving checkpoints for 2950 into models/rnn/model.ckpt.
INFO:tensorflow:Loss for final step: 0.00640675.
Tensor("rnn1/rnn1/transpose:0", shape=(?, 100, 500), dtype=float32)
INFO:tensorflow:Starting evaluation at 2017-12-07-08:58:06
INFO:tensorflow:Restoring parameters from models/rnn/model.ckpt-2950
INFO:tensorflow:Finished evaluation at 2017-12-07-08:58:17
INFO:tensorflow:Saving dict for global step 2950: accuracy = 0.637809, global_step = 2950, loss = 1.91613
{'accuracy': 0.63780922, 'loss': 1.9161277, 'global_step': 2950}
Tensor("rnn1/rnn1/transpose:0", shape=(128, 100, 500), dtype=float32)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from models/rnn/model.ckpt-2950
INFO:tensorflow:Saving checkpoints for 2951 into models/rnn/model.ckpt.
INFO:tenso

{'accuracy': 0.64045936, 'loss': 1.8626181, 'global_step': 3400}
Tensor("rnn1/rnn1/transpose:0", shape=(128, 100, 500), dtype=float32)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from models/rnn/model.ckpt-3400
INFO:tensorflow:Saving checkpoints for 3401 into models/rnn/model.ckpt.
INFO:tensorflow:loss = 0.00908479, step = 3401
INFO:tensorflow:Saving checkpoints for 3450 into models/rnn/model.ckpt.
INFO:tensorflow:Loss for final step: 0.00879681.
Tensor("rnn1/rnn1/transpose:0", shape=(?, 100, 500), dtype=float32)
INFO:tensorflow:Starting evaluation at 2017-12-07-09:34:16
INFO:tensorflow:Restoring parameters from models/rnn/model.ckpt-3450
INFO:tensorflow:Finished evaluation at 2017-12-07-09:34:26
INFO:tensorflow:Saving dict for global step 3450: accuracy = 0.633392, global_step = 3450, loss = 1.85413
{'accuracy': 0.63339221, 'loss': 1.8541303, 'global_step': 3450}
Tensor("rnn1/rnn1/transpose:0", shape=(128, 100, 500), dtype=float32)
INFO:tensorflow:

INFO:tensorflow:Restoring parameters from models/rnn/model.ckpt-3900
INFO:tensorflow:Finished evaluation at 2017-12-07-10:06:57
INFO:tensorflow:Saving dict for global step 3900: accuracy = 0.587456, global_step = 3900, loss = 1.5428
{'accuracy': 0.58745581, 'loss': 1.5428032, 'global_step': 3900}
Tensor("rnn1/rnn1/transpose:0", shape=(128, 100, 500), dtype=float32)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from models/rnn/model.ckpt-3900
INFO:tensorflow:Saving checkpoints for 3901 into models/rnn/model.ckpt.
INFO:tensorflow:loss = 0.110626, step = 3901
INFO:tensorflow:Saving checkpoints for 3950 into models/rnn/model.ckpt.
INFO:tensorflow:Loss for final step: 0.0165897.
Tensor("rnn1/rnn1/transpose:0", shape=(?, 100, 500), dtype=float32)
INFO:tensorflow:Starting evaluation at 2017-12-07-10:10:24
INFO:tensorflow:Restoring parameters from models/rnn/model.ckpt-3950
INFO:tensorflow:Finished evaluation at 2017-12-07-10:10:35
INFO:tensorflow:Saving dict

INFO:tensorflow:Saving checkpoints for 4400 into models/rnn/model.ckpt.
INFO:tensorflow:Loss for final step: 0.00749625.
Tensor("rnn1/rnn1/transpose:0", shape=(?, 100, 500), dtype=float32)
INFO:tensorflow:Starting evaluation at 2017-12-07-10:42:56
INFO:tensorflow:Restoring parameters from models/rnn/model.ckpt-4400
INFO:tensorflow:Finished evaluation at 2017-12-07-10:43:07
INFO:tensorflow:Saving dict for global step 4400: accuracy = 0.641343, global_step = 4400, loss = 1.96523
{'accuracy': 0.64134276, 'loss': 1.9652315, 'global_step': 4400}
Tensor("rnn1/rnn1/transpose:0", shape=(128, 100, 500), dtype=float32)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from models/rnn/model.ckpt-4400
INFO:tensorflow:Saving checkpoints for 4401 into models/rnn/model.ckpt.
INFO:tensorflow:loss = 0.00728796, step = 4401
INFO:tensorflow:Saving checkpoints for 4450 into models/rnn/model.ckpt.
INFO:tensorflow:Loss for final step: 0.00703521.
Tensor("rnn1/rnn1/transpose:0"

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from models/rnn/model.ckpt-4850
INFO:tensorflow:Saving checkpoints for 4851 into models/rnn/model.ckpt.
INFO:tensorflow:loss = 0.0203977, step = 4851
INFO:tensorflow:Saving checkpoints for 4900 into models/rnn/model.ckpt.
INFO:tensorflow:Loss for final step: 0.0123048.
Tensor("rnn1/rnn1/transpose:0", shape=(?, 100, 500), dtype=float32)
INFO:tensorflow:Starting evaluation at 2017-12-07-11:18:57
INFO:tensorflow:Restoring parameters from models/rnn/model.ckpt-4900
INFO:tensorflow:Finished evaluation at 2017-12-07-11:19:07
INFO:tensorflow:Saving dict for global step 4900: accuracy = 0.628975, global_step = 4900, loss = 1.87538
{'accuracy': 0.62897527, 'loss': 1.8753844, 'global_step': 4900}
Tensor("rnn1/rnn1/transpose:0", shape=(128, 100, 500), dtype=float32)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from models/rnn/model.ckpt-4900
INFO:tensorflow:Saving checkpoints for 4

In [ ]:
# def get_batch(path, batch_size):
#     filenames = glob.glob(os.path.join(path, '*npy'))
#     return random.sample(filenames, batch_size)

# def get_batch_max_len(batch_files):
#     batch_max_len=0
#     for f in batch_files:
#         batch_max_len=max(batch_max_len, np.load(f).shape[0])
#     return batch_max_len

# def get_reference(reference_path, reference):
#     with open(reference_path) as csvfile:
#         spamreader=csv.reader(csvfile, delimiter=',', quotechar='|')
#         for row in spamreader:
#             reference[row[0]]=row[1]

# def generate_batch(path):
#     batch_files=get_batch(path, batch_size)
   
#     max_seq=get_batch_max_len(batch_files)

#     batchX=np.zeros((batch_size, 1, max_seq), dtype='float32')
#     i=0
#     for f in batch_files:
#         arr=np.load(f)
#         arr=np.asmatrix(arr)
#         batchX[i][0][0:arr.shape[1]]=arr

#     i=0
#     batchY=np.zeros(batch_size, dtype='int32')
#     for f in batch_files:
#         batchY[i] = reference_train[f.split('/')[-1].split('.')[0]]
#         i += 1
    
#     # reshape batchX
#     batchX = np.swapaxes(batchX, 1, 2)
#     batchX = np.swapaxes(batchX, 0, 1)
    
#     # convert batchY to onehot
#     y_onehot = np.zeros((batch_size, 2))
#     y_onehot[np.where(batchY==1)] = np.array([0,1])
#     y_onehot[np.where(batchY==-1)] = np.array([1,0])
    
#     print('batchX, y_onehot, sizes', batchX.shape, y_onehot.shape)
    
#     return batchX, y_onehot

In [ ]:
# # define variables
# train_path='./data/td/training/'
# validation_path='./data/td/validation/'
# reference_train_path=train_path + 'REFERENCE.csv'
# reference_validation_path=validation_path + 'REFERENCE.csv'
# train_dirs=os.listdir(train_path)
# train_size=len(train_dirs)
# batch_size=20

# batch_train_files=random.sample(os.listdir(train_path), batch_size)

In [ ]:
# hm_epochs = 5
# n_classes = 2
# state_size = 3
# n_units = 30

# x_placeholder = tf.placeholder('float32', [None, batch_size, 1])
# y_placeholder = tf.placeholder('int32', [batch_size, n_classes])


In [ ]:
# def find_maxlength_file(data_dir):
#     data_filenames = glob.glob(os.path.join(data_dir, '*npy'))
#     random.shuffle(data_filenames)

#     num_examples = len(data_filenames)

#     max_length = 0
#     max_length_fn = '_'
#     for i, df in enumerate(data_filenames):
#         length = np.load(df).shape[0]
#         if length > max_length:
#             max_length = length
#             max_length_df = df
            
#     return max_length, max_length_df

# data_dir = '../deeplearning/data/td/training'

# ml, ml_df = find_maxlength_file(data_dir)
# print('ML: {}'.format(ml))
# print('ML file: {}'.format(ml_df))